# Word Embeddings : le modèle Word2Vec

## Imports

In [3]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

488

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 26min 1s, sys: 17.1 s, total: 26min 18s
Wall time: 10min 21s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [15]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [4]:
model = Word2Vec.load("../data/newspapers.model")

### Calculer la similarité entre deux termes

In [36]:
model.wv.similarity("belle", "ravissante")

0.8103219

In [33]:
model.wv.similarity("femme", "ravissante")

0.47960347

In [49]:
model.wv.similarity("homme", "ravissant")

0.14339742

In [35]:
model.wv.similarity("blonde", "ravissante")

0.8873249

In [37]:
model.wv.similarity("rousse", "ravissante")

0.8173834

In [38]:
model.wv.similarity("brune", "ravissante")

0.6901406

In [39]:
model.wv.similarity("ravissante", "naive")

0.7705526

In [40]:
model.wv.similarity("blonde", "naive")

0.71863866

In [50]:
model.wv.similarity("blond", "naif")

0.76688427

In [41]:
model.wv.similarity("brune", "naive")

0.35653964

In [51]:
model.wv.similarity("brun", "naif")

0.43951243

In [42]:
model.wv.similarity("rousse", "naive")

0.6058221

In [52]:
model.wv.similarity("roux", "naif")

0.25419548

### Chercher les mots les plus proches d'un terme donné

In [54]:
model.wv.most_similar("belle", topn=10)

[('jolie', 0.9007665514945984),
 ('petite', 0.87130206823349),
 ('grando', 0.8462585806846619),
 ('coquette', 0.8430774211883545),
 ('petito', 0.828839898109436),
 ('bello', 0.8254029750823975),
 ('ravissante', 0.8103218078613281),
 ('somptueuse', 0.8082084655761719),
 ('fleurie', 0.7998606562614441),
 ('tres_jolie', 0.7905054092407227)]

In [39]:
model.wv.most_similar("ravissante", topn=10)

[('trepidante', 0.8902712464332581),
 ('somptueuse', 0.8888676762580872),
 ('fleurie', 0.8887552618980408),
 ('blonde', 0.887324869632721),
 ('delicieuse', 0.8858428597450256),
 ('gracieuse', 0.877484142780304),
 ('savoureuse', 0.875773549079895),
 ('pimpante', 0.8720426559448242),
 ('eblouissante', 0.8708234429359436),
 ('exquise', 0.8631389737129211)]

In [38]:
model.wv.most_similar("blonde", topn=10)

[('rousse', 0.9025325179100037),
 ('ravissante', 0.8873249292373657),
 ('mignonne', 0.8782892227172852),
 ('svelte', 0.8625671863555908),
 ('douce', 0.8565280437469482),
 ('brune', 0.8554239869117737),
 ('reveuse', 0.849934458732605),
 ('vetue', 0.8485318422317505),
 ('habillee', 0.8480935096740723),
 ('moustache', 0.8461067080497742)]

In [40]:
model.wv.most_similar("brune", topn=10)

[('blonde', 0.8554240465164185),
 ('rousse', 0.8104218244552612),
 ('culotte', 0.8047497868537903),
 ('bleue', 0.8025820255279541),
 ('rayee', 0.8022409081459045),
 ('drapee', 0.7857430577278137),
 ('pelerine', 0.7850284576416016),
 ('soie_noire', 0.7801920771598816),
 ('chcviott', 0.7785747647285461),
 ('fine', 0.769081175327301)]

In [55]:
model.wv.most_similar("rousse", topn=10)

[('blonde', 0.9025324583053589),
 ('rose', 0.8819923400878906),
 ('moustache', 0.8788085579872131),
 ('vipere', 0.8641492128372192),
 ('chatte', 0.8564072251319885),
 ('liane', 0.8518047332763672),
 ('menue', 0.832155704498291),
 ('levre', 0.830085277557373),
 ('cassee', 0.8253190517425537),
 ('berce', 0.8218207955360413)]

In [56]:
model.wv.most_similar("naive", topn=10)

[('divine', 0.9399339556694031),
 ('hautaine', 0.9391617774963379),
 ('souriante', 0.9344111084938049),
 ('instinctive', 0.9268401265144348),
 ('innocente', 0.9232600927352905),
 ('sentimentale', 0.9223591089248657),
 ('touchante', 0.9209133386611938),
 ('douloureuse', 0.9207254648208618),
 ('cruelle', 0.9184885025024414),
 ('poignante', 0.9174149036407471)]

### Faire des recherches complexes à travers l'espace vectoriel

In [14]:
print(model.wv.most_similar(positive=['blonde', 'ravissante'], negative=['fille']))

[('savoureuse', 0.8534754514694214), ('somptueuse', 0.8488709926605225), ('fleurie', 0.8451348543167114), ('trepidante', 0.8439614772796631), ('pimpante', 0.8408786654472351), ('transparente', 0.8298086524009705), ('fraiche', 0.8281316161155701), ('merveilleuse', 0.8214274644851685), ('silencieuse', 0.8197795748710632), ('doree', 0.8177288174629211)]
